In [3]:
import numpy as np
import pandas as pd
import string
import random
from collections import defaultdict, deque
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split as tts
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [4]:
# Load the data
data = pd.read_csv('processed.csv')
y = data['strength'].values
x = data[['length', 'capital', 'small', 'special', 'numeric']].values

# Split the data into training and testing sets
xtrain, xtest, ytrain, ytest = tts(x, y, test_size=0.2, random_state=42)

# Scale the data
sc = StandardScaler()
xtrain = sc.fit_transform(xtrain)
xtest = sc.transform(xtest)

# Train the Random Forest Classifier
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(xtrain, ytrain)

RandomForestClassifier(random_state=42)

In [6]:
# Evaluate the model on the training set
ytrain_pred = model.predict(xtrain)
train_accuracy = accuracy_score(ytrain, ytrain_pred)
train_report = classification_report(ytrain, ytrain_pred)
train_matrix = confusion_matrix(ytrain, ytrain_pred)

print("Training Accuracy:", train_accuracy)
print("Training Classification Report:\n", train_report)
print("Training Confusion Matrix:\n", train_matrix)

# Evaluate the model on the testing set
ytest_pred = model.predict(xtest)
test_accuracy = accuracy_score(ytest, ytest_pred)
test_report = classification_report(ytest, ytest_pred)
test_matrix = confusion_matrix(ytest, ytest_pred)

print("Testing Accuracy:", test_accuracy)
print("Testing Classification Report:\n", test_report)
print("Testing Confusion Matrix:\n", test_matrix)

Training Accuracy: 1.0
Training Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00     71793
           1       1.00      1.00      1.00    397282
           2       1.00      1.00      1.00     66636

    accuracy                           1.00    535711
   macro avg       1.00      1.00      1.00    535711
weighted avg       1.00      1.00      1.00    535711

Training Confusion Matrix:
 [[ 71793      0      0]
 [     0 397282      0]
 [     0      0  66636]]
Testing Accuracy: 1.0
Testing Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00     17908
           1       1.00      1.00      1.00     99519
           2       1.00      1.00      1.00     16501

    accuracy                           1.00    133928
   macro avg       1.00      1.00      1.00    133928
weighted avg       1.00      1.00      1.00    133928

Testing Confusion Matrix:
 

In [5]:
class PasswordGenerator:
    def __init__(self, password_length, charset=string.ascii_letters + string.digits + string.punctuation, epsilon=0.5, epsilon_min=0.01, epsilon_decay=0.999, gamma=0.95, alpha=1e-5, batch_size=32, memory_size=1000):
        self.password_length = password_length
        self.charset = charset
        self.q_table = defaultdict(lambda: np.zeros(len(self.charset)))
        self.epsilon = epsilon
        self.epsilon_min = epsilon_min
        self.epsilon_decay = epsilon_decay
        self.gamma = gamma
        self.alpha = alpha
        self.batch_size = batch_size
        self.memory = deque(maxlen=memory_size)

    def get_action(self, state):
        q_values = self.q_table[state]
        exp_q_values = np.exp(q_values / self.epsilon)
        probabilities = exp_q_values / np.sum(exp_q_values)
        probabilities = np.array(probabilities)  # Convert probabilities to a numpy array
        return self.charset[np.random.choice(len(self.charset), p=probabilities)]

    def update_q_table(self, state, action, reward, next_state):
        action_index = self.charset.index(action)
        current_q = self.q_table[state][action_index]
        target_q = reward + self.gamma * np.max(self.q_table[next_state])
        self.q_table[state][action_index] = current_q + self.alpha * (target_q - current_q)

    def get_reward(self, guess, target_password):
        if guess == target_password:
            return 1
        else:
            # Calculate the reward based on the number of letters, uppercase, lowercase, special characters, and digits
            num_letters = sum(1 for char in guess if char in string.ascii_letters)
            num_uppercase = sum(1 for char in guess if char in string.ascii_uppercase)
            num_lowercase = sum(1 for char in guess if char in string.ascii_lowercase)
            num_special = sum(1 for char in guess if char in string.punctuation)
            num_digits = sum(1 for char in guess if char in string.digits)

            # Assign higher rewards for passwords with the desired characteristics
            reward = -0.1
            if num_letters == self.password_length:
                reward += 0.15
            if num_uppercase >= 1:
                reward += 0.15
            if num_lowercase >= 1:
                reward += 0.15
            if num_special >= 1:
                reward += 0.15
            if num_digits >= 1:
                reward += 0.1

            return reward

    def train(self, target_passwords, num_episodes, checkpoint=100):
        for episode in range(num_episodes):
            target_password = random.choice(target_passwords)
            state = ''
            for _ in range(self.password_length):
                action = self.get_action(state)
                reward = self.get_reward(state + action, target_password)
                next_state = state + action
                self.memory.append((state, action, reward, next_state, next_state == target_password))
                self.update_q_table(state, action, reward, next_state)
                state = next_state

            if self.epsilon > self.epsilon_min:
                self.epsilon *= self.epsilon_decay

            if (episode + 1) % checkpoint == 0:
                print(f"Episode {episode + 1}/{num_episodes}, Epsilon: {self.epsilon:.3f}")

    def generate_password(self, target_password):
        candidate_passwords = []
        for _ in range(5):  # Generate 5 candidate passwords
            state = ''
            for _ in range(self.password_length):
                action = self.get_action(state)
                state += action
            candidate_passwords.append(state)

        return random.choice(candidate_passwords)

In [7]:
# Example usage
target_passwords = ['Tr0ub4dor&3', 'correcthorsebatterystaple#$', 'Th1s1sAStr0ngP@ssw0rd', 'Wr0ngP@ssw0rd123*']
password_generator = PasswordGenerator(password_length=16)
password_generator.train(target_passwords, num_episodes=10000)

num_passwords = 5
for _ in range(num_passwords):
    generated_password = password_generator.generate_password(random.choice(target_passwords))
    print(f"Generated password: {generated_password}")

    # Classify the generated password using the Random Forest Classifier
    password_features = np.array([len(generated_password),
                                  sum(1 for char in generated_password if char in string.ascii_uppercase),
                                  sum(1 for char in generated_password if char in string.ascii_lowercase),
                                  sum(1 for char in generated_password if char in string.punctuation),
                                  sum(1 for char in generated_password if char in string.digits)])
    password_features = sc.transform([password_features])
    password_class = model.predict(password_features)[0]
    print(f"Password strength: {['Weak', 'Medium', 'Strong'][password_class]}")

# Test the accuracy of the RL model
num_test_passwords = 1000
correct_predictions = 0
for _ in range(num_test_passwords):
    generated_password = password_generator.generate_password(random.choice(target_passwords))
    password_features = np.array([len(generated_password),
                                  sum(1 for char in generated_password if char in string.ascii_uppercase),
                                  sum(1 for char in generated_password if char in string.ascii_lowercase),
                                  sum(1 for char in generated_password if char in string.punctuation),
                                  sum(1 for char in generated_password if char in string.digits)])
    password_features = sc.transform([password_features])
    predicted_class = model.predict(password_features)[0]

    # Define your criteria for weak, medium, and strong passwords
    if predicted_class == 0 and len(generated_password) < 8:
        correct_predictions += 1
    elif predicted_class == 1 and (len(generated_password) >= 8 and len(generated_password) < 12):
        correct_predictions += 1
    elif predicted_class == 2 and len(generated_password) >= 12:
        correct_predictions += 1

accuracy = correct_predictions / num_test_passwords
print(f"Accuracy of the RL model: {accuracy:.2f}")

Episode 100/10000, Epsilon: 0.452
Episode 200/10000, Epsilon: 0.409
Episode 300/10000, Epsilon: 0.370
Episode 400/10000, Epsilon: 0.335
Episode 500/10000, Epsilon: 0.303
Episode 600/10000, Epsilon: 0.274
Episode 700/10000, Epsilon: 0.248
Episode 800/10000, Epsilon: 0.225
Episode 900/10000, Epsilon: 0.203
Episode 1000/10000, Epsilon: 0.184
Episode 1100/10000, Epsilon: 0.166
Episode 1200/10000, Epsilon: 0.151
Episode 1300/10000, Epsilon: 0.136
Episode 1400/10000, Epsilon: 0.123
Episode 1500/10000, Epsilon: 0.111
Episode 1600/10000, Epsilon: 0.101
Episode 1700/10000, Epsilon: 0.091
Episode 1800/10000, Epsilon: 0.083
Episode 1900/10000, Epsilon: 0.075
Episode 2000/10000, Epsilon: 0.068
Episode 2100/10000, Epsilon: 0.061
Episode 2200/10000, Epsilon: 0.055
Episode 2300/10000, Epsilon: 0.050
Episode 2400/10000, Epsilon: 0.045
Episode 2500/10000, Epsilon: 0.041
Episode 2600/10000, Epsilon: 0.037
Episode 2700/10000, Epsilon: 0.034
Episode 2800/10000, Epsilon: 0.030
Episode 2900/10000, Epsilon: 